In [1]:
import json
import openai
import datetime
from langame import LangameClient
c = LangameClient(path_to_config_file="../config.yaml")

In [2]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("generation")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

/Users/louisbeaumont/Documents/langame-worker/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipping id: 0ZcPZDcvuqgialj2wHpL, garbage
Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: 4sIjdMJr0exliT9yfvuL, garbage
Skipping id: 5MAMBAYySyu3lKrR3zLt, garbage
Skipping id: 6jNOw0e7VA6lHWddtqMy, garbage
Skipping id: 7yHXwGPlHpopQAHU3xf1, garbage
Skipping id: 8e9jVhvFpUIzj7ocNv06, garbage
Skipping id: 8tzYKdv2Hkwxsm8eNQgj, garbage
Skipping id: 9NdMRn8C285Eu3k4PdsS, garbage
Skipping id: 9ZfEEkrB3cjLooW6YZM2, garbage
Skipping id: B8hRGbGGOweqEyxuuplE, garbage
Skipping id: BH9ZtGe1g6q3KmK7oV2f, garbage
Skipping id: BZjXMPPKlmcqXG23i0mI, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: EBZCFk0aZPVS1SinJwcz, garbage
Skipping id: EPBBhoxNvsJ7XdTc

Using 8 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 09/18/2022, 15:53:01
There are 2922 embeddings of dim 768
	Compute estimated construction time of the index 09/18/2022, 15:53:01
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0001 secs
	Checking that your have enough memory available to create the index 09/18/2022, 15:53:01
10.7MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0009 secs
	Selecting most promising index types given data characteristics 09/18/2022, 15:53:01
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 09/18/2022, 15:53:01
		-> Instanciate the index HNSW15 09/18/2022, 15:53:01
		>>> Finished "-> Instanciate the index HNSW15" in 0.003

100%|██████████| 1/1 [00:00<00:00, 263.66it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.1068 secs
	>>> Finished "Creating the index" in 0.1121 secs
	Computing best hyperparameters 09/18/2022, 15:53:01


	>>> Finished "Computing best hyperparameters" in 2.9018 secs
The best hyperparameters are: efSearch=16383
	Compute fast metrics 09/18/2022, 15:53:04
1456
	>>> Finished "Compute fast metrics" in 10.0316 secs
	Saving the index on local disk 09/18/2022, 15:53:14
	>>> Finished "Saving the index on local disk" in 0.0086 secs
Recap:
{'99p_search_speed_ms': 8.801004049998088,
 'avg_search_speed_ms': 6.867052949862856,
 'compression ratio': 0.9576711662625687,
 'index_key': 'HNSW15',
 'index_param': 'efSearch=16383',
 'nb vectors': 2922,
 'reconstruction error %': 0.0,
 'size in bytes': 9373138,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 13.0729 secs


In [4]:
out_file_name = f"../data/fine_tune_generation_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ###",
            "completion": f" {e['content']}\n",
        }, outfile)
        outfile.write('\n')

In [5]:
!head -n3 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "personal,relationship,relationships,social,big talk,personal growth ###",
  "completion": " Give eachother four praises and one critique\n"
}
{
  "prompt": "ice breaker ###",
  "completion": " When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new?\n"
}
{
  "prompt": "space exploration,space travel ###",
  "completion": " Do you think humans are the only intelligent life in the universe?\n"
}


In [6]:
!echo "execute: openai tools fine_tunes.prepare_data -f $(pwd)/$out_file_name"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
execute: openai tools fine_tunes.prepare_data -f /Users/louisbeaumont/Documents/langame-worker/notebooks/../data/fine_tune_generation_18_09_2022.jsonl


In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
train, test = train_test_split(
    open(f"{out_file_name.replace('.jsonl', '_prepared.jsonl')}").read().splitlines(), test_size=0.1)
with open(f"{out_file_name.replace('.jsonl', '')}_prepared_train.jsonl", "w") as outfile:
    outfile.write("\n".join(train))
with open(f"{out_file_name.replace('.jsonl', '')}_prepared_test.jsonl", "w") as outfile:
    outfile.write("\n".join(test))

In [8]:
!head -n3 {out_file_name.replace('.jsonl', '')}_prepared_train.jsonl | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "transhumanism,public welfare ###",
  "completion": " What do you think will happen if we continue to push the technological and social limits of humanity?\n"
}
{
  "prompt": "ice breaker ###",
  "completion": " What genre would your life story be if it was turned into a movie?\n"
}
{
  "prompt": "ice breaker,emotions ###",
  "completion": " If you could condense your own thoughts and feelings down to one word, what would that word be? Why?\n"
}


In [13]:
train_file = openai.File.create(
  file=open(f"{out_file_name.replace('.jsonl', '')}_prepared_train.jsonl", "rb"),
  purpose="fine-tune"
)
valid_file = openai.File.create(
  file=open(f"{out_file_name.replace('.jsonl', '')}_prepared_test.jsonl", "rb"),
  purpose="fine-tune"
)
ft = openai.FineTune.create(
    training_file=train_file["id"],
    validation_file=valid_file["id"],
    model="curie",
    suffix="generation",
)

In [19]:
from openai.wandb_logger import WandbLogger
import wandb
import re
values = open("../.env.production", "r").read()
key = re.findall(r"WANDB_KEY=\"(.*)\"", values)[0]
wandb.login(key=key, relogin=True)
WandbLogger.sync(
    id=ft["id"],
    project="generation",
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/louisbeaumont/.netrc


Fine-tune ft-x3ojk3Kxs2Gkz9SJhE4m1fK6 has the status "running" and will not be logged


'🎉 wandb sync completed successfully'

In [5]:
results = openai.FineTune.retrieve(
    ft["id"],
)
results

NameError: name 'ft' is not defined

In [8]:
!rm -rf _prepared*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
